In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
train = pd.read_csv('train_Minterpolate.csv')
train

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,v01,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
0,A,5,1,9,STN001,3,A,5,1,12,0,0,0,0,0,0,0,0,0,0.0,0
1,A,5,1,9,STN001,6,A,5,1,15,0,0,0,0,0,0,0,0,0,0.0,0
2,A,5,1,9,STN001,9,A,5,1,18,0,0,0,0,0,0,0,0,0,0.0,0
3,A,5,1,9,STN001,12,A,5,1,21,0,0,0,0,0,0,0,0,0,0.0,0
4,A,5,1,9,STN001,15,A,5,2,0,0,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457247,C,9,30,21,STN020,228,C,10,10,9,9,7,2,2,0,0,0,0,0,0.5,3
1457248,C,9,30,21,STN020,231,C,10,10,12,13,9,5,3,1,0,0,0,0,0.0,0
1457249,C,9,30,21,STN020,234,C,10,10,15,13,9,5,3,1,0,0,0,0,1.9,4
1457250,C,9,30,21,STN020,237,C,10,10,18,8,6,4,2,2,0,0,0,0,1.9,4


In [2]:
df = train.iloc[:, :19]

df['ef_year_temp'] = df['ef_year'].replace({'A': '2020', 'B': '2021', 'C': '2022'})
df['ef_datetime'] = pd.to_datetime(df[['ef_year_temp', 'ef_month', 'ef_day', 'ef_hour']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d-%H')

df['weight'] = 1 / df['dh'] # (dh가 작으면 예측 날짜와 가깝다는 것을 의미하니까 정확도가 올라간다는 가정으로) 가중치 부여 

for var in ['v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09']:
    df[f'{var}_weighted'] = df[var] * df['weight']

# 가중치 평균
weighted_df = df.groupby(['stn4contest', 'ef_datetime']).apply(
    lambda x: pd.Series({
        'v01': np.average(x['v01'], weights=x['weight']),
        'v02': np.average(x['v02'], weights=x['weight']),
        'v03': np.average(x['v03'], weights=x['weight']),
        'v04': np.average(x['v04'], weights=x['weight']),
        'v05': np.average(x['v05'], weights=x['weight']),
        'v06': np.average(x['v06'], weights=x['weight']),
        'v07': np.average(x['v07'], weights=x['weight']),
        'v08': np.average(x['v08'], weights=x['weight']),
        'v09': np.average(x['v09'], weights=x['weight'])
    })
).reset_index()

train_y = train.groupby(['stn4contest', 'ef_year', 'ef_month', 'ef_day', 'ef_hour'])[['vv', 'class_interval']].mean().reset_index(drop=True)

weighted_df['vv'] = train_y['vv']
weighted_df['class_interval'] = train_y['class_interval']
weighted_df = weighted_df.astype({'class_interval': 'int'})

weighted_df['year'] = weighted_df['ef_datetime'].dt.year
weighted_df['month'] = weighted_df['ef_datetime'].dt.month
weighted_df['day'] = weighted_df['ef_datetime'].dt.day
weighted_df['hour'] = weighted_df['ef_datetime'].dt.hour

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
stn_label = encoder.fit_transform(weighted_df['stn4contest'])
weighted_df['stn4contest'] = stn_label
weighted_df = weighted_df.drop(columns=['ef_datetime'])



data = weighted_df
data

,stn4contest,v01,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval,year,month,day,hour
0,0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,12
1,0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,15
2,0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,18
3,0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,21
4,0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,2020,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77995,19,10.948718,7.974359,3.948718,2.0,0.487179,0.0,0.0,0.0,0.0,0.5,3,2022,10,10,9
77996,19,13.000000,9.000000,5.000000,3.0,1.000000,0.0,0.0,0.0,0.0,0.0,0,2022,10,10,12
77997,19,13.000000,9.000000,5.000000,3.0,1.000000,0.0,0.0,0.0,0.0,1.9,4,2022,10,10,15
77998,19,8.000000,6.000000,4.000000,2.0,2.000000,0.0,0.0,0.0,0.0,1.9,4,2022,10,10,18


In [3]:
from pycaret.regression import *  
regression_setup = setup(
    data=weighted_df.drop(columns=['class_interval']), 
    target='vv',  # 목표 변수 설정
    numeric_features=['v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09'],  # 수치형 특징 설정
    categorical_features=['stn4contest', 'year', 'month', 'day', 'hour'],  # 범주형 특징 설정
    session_id=123,  # 재현성을 위한 세션 ID 설정
    use_gpu=True,  # GPU 사용
    fold_strategy='kfold',  # K-Fold 교차 검증 사용
    fold=10,  # Fold 수 설정
    verbose=False  # 출력을 최소화
)

In [4]:
weighted_df.head()

,stn4contest,v01,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval,year,month,day,hour
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,12
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,15
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,18
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,21
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,2,0


In [5]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7783,11.1704,3.3207,0.4061,0.4237,2.8939,1.7810
rf,Random Forest Regressor,0.8150,11.5781,3.3814,0.3842,0.4523,3.0813,38.6020
br,Bayesian Ridge,0.9299,11.7564,3.4083,0.3741,0.4911,3.6420,1.3100
lr,Linear Regression,0.9383,11.7578,3.4086,0.3739,0.4939,3.7795,1.3130
ridge,Ridge Regression,0.9383,11.7577,3.4086,0.3739,0.4939,3.7792,1.1760
en,Elastic Net,0.8938,11.9893,3.4378,0.3647,0.4814,3.6189,1.2980
gbr,Gradient Boosting Regressor,0.8030,12.0551,3.4488,0.3597,0.4304,3.0676,49.2210
llar,Lasso Least Angle Regression,0.8927,12.1439,3.4583,0.3578,0.4812,3.5765,1.2010
lasso,Lasso Regression,0.8927,12.1440,3.4583,0.3578,0.4812,3.5765,1.4210
omp,Orthogonal Matching Pursuit,0.9000,12.1336,3.4584,0.3572,0.4823,3.5636,1.1740


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [7]:
tuned_model = tune_model(best_model, n_iter=5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8589,18.7197,4.3266,0.3158,0.4408,3.1355
1,0.7609,8.2193,2.8669,0.3969,0.4334,2.6808
2,0.7919,9.6160,3.1010,0.4055,0.4350,2.4931
3,0.8056,14.0382,3.7468,0.3784,0.4201,3.7235
4,0.7532,9.9057,3.1473,0.4255,0.4126,3.2759
5,0.7170,9.0792,3.0132,0.3734,0.4116,2.8750
6,0.7903,13.1033,3.6198,0.3964,0.4145,3.7741
7,0.7465,8.8698,2.9782,0.4321,0.4259,2.2535
8,0.8397,13.0758,3.6161,0.4152,0.4323,2.5787


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [8]:
save_model(tuned_model, 'best_rainfall_prediction_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['v01', 'v02', 'v03', 'v04', 'v05',
                                              'v06', 'v07', 'v08', 'v09'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['stn4contest', 'year', 'month',
                                              'day', 'hour'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['stn4contest', 'year', 'month',
                                              'hour'],
                                     transformer=OneHotEncoder(cols=['stn4contest',
                                                                     'year',
                                                                     'month',
               

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

test = pd.read_csv('test_stn+ef_label.csv')
submit = pd.read_csv('rainfall_test.csv')

prediction_test = predict_model(tuned_model,  data=test)
idx999 = prediction_test[prediction_test['class_interval'].notnull()].index
print(idx999)

Index([44, 1198, 2024, 2092, 2093, 3971, 5881], dtype='int64')


In [17]:
#prediction_train['prediction_cl'] = prediction_train['prediction_label'].apply(classify_class)
prediction_test['class_interval'] = prediction_test['prediction_label'].apply(classify_class)
prediction_test.loc[idx999, 'class_interval'] = -999

#rainfall_test. 제거 데이터
submit=pd.read_csv("rainfall_test2.csv")
#제출용 데이터
realsubmit=pd.read_csv("rainfall_test.csv")
#submit-prediction 지점,date 맞추기
prediction_test['ef_datetime'] = pd.to_datetime(prediction_test['ef_datetime'])
submit['ef_year_temp'] = submit['ef_year'].replace({'D': '2023'})
submit['ef_datetime'] = pd.to_datetime(submit[['ef_year_temp', 'ef_month', 'ef_day', 'ef_hour']].astype(str).agg('-'.join, axis=1), 
                                       format='%Y-%m-%d-%H')
stn_mapping = {'STN031': 31, 'STN032': 32, 'STN033': 33, 'STN034': 34, 'STN035': 35}
submit['stn4contest'] = submit['stn4contest'].replace(stn_mapping)
#prediction의 class_interval submit에 흩뿌리기
# prediction_test=new_data.copy() #new_data : 개개인마다 달라유
merged = submit.merge(prediction_test[['ef_datetime', 'stn4contest', 'class_interval']], 
                      on=['ef_datetime', 'stn4contest'], 
                      how='left', 
                      suffixes=('', '_pred'))
merged['class_interval_pred']
#-999 index 저장
idx999 = realsubmit[realsubmit['rainfall_test.class_interval'].notnull()].index
merged.loc[idx999, 'class_interval_pred'] = -999
#제출용 데이터에 class_interval 채우기
realsubmit['rainfall_test.class_interval']=merged['class_interval_pred']
#저장
realsubmit.to_csv("240494.csv")  # 각자 접수번호.csv 로 저장

In [19]:
realsubmit['rainfall_test.class_interval'].value_counts()

rainfall_test.class_interval
 0      77327
 2       8935
 5       8297
 1       6977
 4       6618
 3       6409
 6       4298
 7       2269
 8        580
 9        160
-999      130
Name: count, dtype: int64

In [12]:
def classify_class(rainfall):
    if rainfall < 0.1:
        return 0
    elif 0.1 <= rainfall < 0.2:
        return 1
    elif 0.2 <= rainfall < 0.5:
        return 2
    elif 0.5 <= rainfall < 1.0:
        return 3
    elif 1.0 <= rainfall < 2.0:
        return 4
    elif 2.0 <= rainfall < 5.0:
        return 5
    elif 5.0 <= rainfall < 10.0:
        return 6
    elif 10.0 <= rainfall < 20.0:
        return 7
    elif 20.0 <= rainfall < 30.0:
        return 8
    else:
        return 9
